In [1]:
import numpy as np
import math
import scipy.io
import os
import matplotlib.pyplot as plt
from scipy.io import loadmat
import ipywidgets as widgets
import torch

In [10]:
def create_spatial_grid(stim, stim_ecc, gridsize):
    ecc = np.linspace(-stim_ecc,stim_ecc,gridsize)
    Y, X = np.meshgrid(ecc, ecc, indexing='ij')
    Y = -1*Y
    (s1, s2) = (stim.shape[0] // X.shape[0]+1, stim.shape[1] // X.shape[0]+1)
    input_stim = stim[::s1,::s2]
    return input_stim, X, Y

In [ ]:
def flat_gaussian_field(X,Y,x,y,sigma,gain,normalize):
    gaussian = gain*np.exp(-((X-x)**2 +(Y-y)**2)/(2*sigma)**2)
    gaussian =  np.reshape(gaussian, (len(X)*len(X)))
    if normalize: # this normalizes the Gaussian field to the unit volume before flattening it
        gaussian = gaussian/np.linalg.norm(gaussian)
    return gaussian

In [ ]:
def create_prf_params(n_voxels, stim_ecc, X, Y, sigmascaling):
    coord = np.sqrt(((stim_ecc)**2)/2)
    nCenters = int(np.sqrt(n_voxels))
    x = -1*np.linspace(-coord, coord, nCenters)
    y = np.linspace(-coord, coord, nCenters)
    prf_parameters = np.zeros((4,n_voxels))
    sigma = np.zeros((n_voxels))
    gain = np.zeros((n_voxels))

    iter_idx = 0
    for i in range(0,len(x)):
        for j in range(0,len(y)):
            prf_parameters[0,iter_idx] = x[i] 
            prf_parameters[1,iter_idx] = y[j]
            if sigmascaling == 'eccentric':
                prf_parameters[2,iter_idx] = 0.05 + 0.2*(np.sqrt(x[i]**2 +  y[j]**2)) # sigma
            elif sigmascaling == 'equal':
                prf_parameters[2,iter_idx] = 0.03
            prf_parameters[3,iter_idx] = 1 # assume uniform voxel gain for simplicity
            iter_idx = iter_idx + 1 

    return prf_parameters

In [ ]:
def simulate_normalization_model(stimpath, stim_ecc, attx0, atty0, attsd, attgain, suppWeight, summWeight, sigmaNorm, gridsize):

    # load the stimulus
    stimtemp = scipy.io.loadmat(stimpath + '/stim.mat')
    stimtemp = stimtemp['stim']
    stimorig = stimtemp[:,:,0:48]

    input_stim, X, Y = create_spatial_grid(stimorig, stim_ecc, gridsize)
    stim = np.reshape(input_stim, (gridsize*gridsize, input_stim.shape[2]))
        # set the anonymous functions for spatial transformations:
    flatten = lambda x: np.reshape(x, (gridsize*gridsize))
    unflatten = lambda x: np.reshape(x, (gridsize,gridsize,stimorig.shape[2]))

        # simulation specific part 
    nCenters = len(input_stim[0])
    prf_parameters = create_prf_params(nCenters*nCenters, stim_ecc, X, Y, 'eccentric')

    simulated_prfs = np.zeros((nCenters*nCenters, nCenters*nCenters))
    for rf in range(0,nCenters*nCenters):
        simulated_prfs[:,rf] = flat_gaussian_field(X,Y,prf_parameters[1,rf],prf_parameters[0,rf],prf_parameters[2,rf], prf_parameters[3,rf], True)

    # preallocate
    stimdrive    = np.zeros((len(simulated_prfs[1]),len(input_stim[1,1,:])))
    numerator = np.zeros((len(simulated_prfs[1]),len(input_stim[1,1,:])))
    surroundresponse = np.zeros((len(simulated_prfs[1]),len(input_stim[1,1,:])))

    for stimidx in range(0,len(input_stim[1,1,:])):
        for rf in range(0,len(simulated_prfs[1])):
            RF = simulated_prfs[:,rf]
            stim = input_stim[:,:,stimidx]
            stim = flatten(stim)
            stimdrive[rf,stimidx] = np.inner(RF,stim)
            attweight = np.exp(-((attx0-prf_parameters[1,rf])**2 +(atty0-prf_parameters[0,rf])**2)/(2*attsd)**2)
            attweight = attgain*attweight+1
            numerator[rf,stimidx] = np.multiply(stimdrive[rf,stimidx],attweight)
        
        for rfsuppind in range(0,len(simulated_prfs[1])):
            distance = np.sqrt((X-prf_parameters[1,rfsuppind])**2+(Y-prf_parameters[0,rfsuppind])**2) 
            # distance is the distance of the center of a given pRF from other pRFs here. This is because each pixel
            # in our image, defined within our meshgrid (X,Y) represents a pRF. Jon last week raised a potential issue
            # here where the 'imperfections' of the pRF coverage will distort the calculations. Let's say there are fewer
            # pRFs surrounding a given pRF. If we replace the X and Y variables of this metric by the center preference
            # of these voxels, a voxel whose pRF is not surrounded by many other pRFs will have 'empty holes'. This will
            # distort the calculation of the suppressive surround. 
            
        
            suppfield = np.exp(-.5*(distance/(prf_parameters[2,rfsuppind]*suppWeight))**2)
            suppfield = suppfield / np.sum(suppfield)
            flatsurr = flatten(suppfield)
            surroundresponse[rfsuppind,stimidx] = np.inner(np.transpose(flatsurr),numerator[:,stimidx])

    predneuralweights = numerator/(surroundresponse + sigmaNorm)
    spsummedresponse = np.zeros((len(prf_parameters[1]),len(input_stim[1,1,:])))

    for stimidx in range(0,len(input_stim[1,1,:])):
        for summidx in range(0,len(simulated_prfs[1])):
            distance = np.sqrt((X-prf_parameters[1,summidx])**2+(Y-prf_parameters[0,summidx])**2);
            summfield = np.exp(-.5*(distance/(prf_parameters[2,summidx]*summWeight))**2)
            summfield = summfield / np.sum(summfield)
            flatsumm = flatten(summfield)
            spsummedresponse[summidx,stimidx] = np.inner(np.transpose(flatsumm),predneuralweights[:,stimidx])

    return numerator, surroundresponse, predneuralweights, spsummedresponse

In [ ]:
stimpath = '/Volumes/server/Projects/attentionpRF/Simulations/matlab_scripts/stimfiles'
stim_ecc    = 10
attgain     = 4
attx0       = 0
atty0       = 5
attsd       = 1
sigmaNorm   = 0.01
gridsize    = 32
suppWeight  = 3
summWeight  = 3
# get the baseline estimates
numerator_base, surroundresponse_base, baseline_neural, spsummed_base = simulate_normalization_model(stimpath, stim_ecc, 0, 0, 2.3, 2, suppWeight, summWeight, sigmaNorm, gridsize)

# get the attention estimates
numerator, surroundresponse, predneuralweights, spsummedresponse = simulate_normalization_model(stimpath, stim_ecc, attx0, atty0, attsd, attgain,suppWeight,summWeight,  sigmaNorm, gridsize)

In [ ]:
spsummedresponse_image = np.reshape(spsummedresponse, (gridsize, gridsize, spsummedresponse.shape[1]))
predneuralweights_image = np.reshape(predneuralweights, (gridsize, gridsize, predneuralweights.shape[1]))

simulated_surround_img = np.reshape(surroundresponse, (gridsize, gridsize, predneuralweights.shape[1]))
numImg = np.reshape(numerator, (gridsize, gridsize, predneuralweights.shape[1]))


@widgets.interact(stimuluslocation=widgets.IntSlider(min=0, max=predneuralweights_image.shape[2]-1, step=1, value=0))
def plot_populationresponse(stimuluslocation):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[10,10])
    ax1.imshow(numImg[:,:,stimuluslocation], cmap='Reds')
    ax1.set_title('numerator')
    ax2.imshow(simulated_surround_img[:,:,stimuluslocation],cmap='Reds')
    ax2.set_title('suppressive drive')

In [ ]:
# visualize

@widgets.interact(stimuluslocation=widgets.IntSlider(min=0, max=predneuralweights_image.shape[2]-1, step=1, value=0))
def plot_populationresponse(stimuluslocation):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[10,10])
    ax1.imshow(predneuralweights_image[:,:,stimuluslocation], cmap='Reds')
    ax1.set_title('Predicted neural responses before summation')
    ax2.imshow(spsummedresponse_image[:,:,stimuluslocation], cmap='Reds')
    ax2.set_title('Predicted neural responses after summation')
